In [2]:
from glob import glob

import torch
from scene import Scene
import os
import open3d as o3d
#from open3d import JVisualizer
import sys
import numpy as np
from tqdm import tqdm
from os import makedirs
from gaussian_renderer import render
import torchvision
from argparse import ArgumentParser, Namespace

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.offline import init_notebook_mode, iplot

from utils.general_utils import safe_state
from arguments import ModelParams, PipelineParams, get_combined_args
from gaussian_renderer import GaussianModel

from scene.colmap_loader import (
    read_extrinsics_text, read_intrinsics_text, qvec2rotmat,
    read_extrinsics_binary, read_intrinsics_binary,
    read_points3D_binary, read_points3D_text
)
from scene.dataset_readers import (
    SceneInfo, readColmapCameras
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
SOURCE_DATA_PATH_LIST = [
    "/home/hjp/KUAICV/NIPS2024_LOCAL/datasets/tandt/train/",
    "/home/hjp/KUAICV/NIPS2024_LOCAL/nerfstudio/nerfstudio/data/color_downsampled/"
]

# read camera parameter files and pointcloud files

SCENE_IDX = 0

SOURCE_DATA_PATH = SOURCE_DATA_PATH_LIST[SCENE_IDX]

cameras_extrinsic_file = os.path.join(SOURCE_DATA_PATH, "sparse/0", "images.bin")
cameras_intrinsic_file = os.path.join(SOURCE_DATA_PATH, "sparse/0", "cameras.bin")
pointcloud_bin_file = os.path.join(SOURCE_DATA_PATH, "sparse/0", "points3D.bin")
pointcloud_ply_file = os.path.join(SOURCE_DATA_PATH, "sparse/0", "points3D.ply")

cam_extrinsics = read_extrinsics_binary(cameras_extrinsic_file)
cam_intrinsics = read_intrinsics_binary(cameras_intrinsic_file)
xyz, rgb, _ = read_points3D_binary(pointcloud_bin_file)

cam_info_list = readColmapCameras(
    cam_extrinsics  = cam_extrinsics,
    cam_intrinsics  = cam_intrinsics,
    images_folder   = os.path.join(SOURCE_DATA_PATH, "images")
)

pcd = o3d.io.read_point_cloud(pointcloud_ply_file)
# get numpber of points
print(len(pcd.points), "points")

Reading camera 301/301
182686 points


In [4]:
from plyfile import PlyData, PlyElement
import math

MODEL_PATH = "/home/hjp/KUAICV/NIPS2024_LOCAL/gaussian-splatting/output/e991fed0-9/"
out_pcd_path = os.path.join(
    MODEL_PATH,
    "point_cloud/iteration_30000/point_clout.ply"
)
out_pcd_path = "/home/hjp/KUAICV/NIPS2024_LOCAL/gaussian-splatting/output/e991fed0-9/point_cloud/iteration_30000/point_clout.ply"
out_pcd_path = "/home/hjp/KUAICV/NIPS2024_LOCAL/gaussian-splatting/output/e991fed0-9/point_cloud/iteration_30000/point_cloud.ply"

ply_data = PlyData.read(out_pcd_path)

xyz = np.stack((
    np.asarray(ply_data.elements[0]["x"]),
    np.asarray(ply_data.elements[0]["y"]),
    np.asarray(ply_data.elements[0]["z"])),
    axis=1
)

opacities = np.asarray(ply_data.elements[0]["opacity"])[..., np.newaxis]

features_dc = np.zeros((xyz.shape[0], 3, 1))
features_dc[:, 0, 0] = np.asarray(ply_data.elements[0]["f_dc_0"])
features_dc[:, 1, 0] = np.asarray(ply_data.elements[0]["f_dc_1"])
features_dc[:, 2, 0] = np.asarray(ply_data.elements[0]["f_dc_2"])

extra_f_names = [p.name for p in ply_data.elements[0].properties if p.name.startswith("f_rest_")]
extra_f_names = sorted(extra_f_names, key = lambda x: int(x.split('_')[-1]))

max_sh_degree = int(math.sqrt((len(extra_f_names) + 3) // 3) - 1)
assert len(extra_f_names)==3*(max_sh_degree + 1) ** 2 - 3
features_extra = np.zeros((xyz.shape[0], len(extra_f_names)))
for idx, attr_name in enumerate(extra_f_names):
    features_extra[:, idx] = np.asarray(ply_data.elements[0][attr_name])
# Reshape (P,F*SH_coeffs) to (P, F, SH_coeffs except DC)

features_extra = features_extra.reshape((features_extra.shape[0], 3, (max_sh_degree + 1) ** 2 - 1))

scale_names = [p.name for p in ply_data.elements[0].properties if p.name.startswith("scale_")]
scale_names = sorted(scale_names, key = lambda x: int(x.split('_')[-1]))
scales = np.zeros((xyz.shape[0], len(scale_names)))
for idx, attr_name in enumerate(scale_names):
    scales[:, idx] = np.asarray(ply_data.elements[0][attr_name])

rot_names = [p.name for p in ply_data.elements[0].properties if p.name.startswith("rot")]
rot_names = sorted(rot_names, key = lambda x: int(x.split('_')[-1]))
rots = np.zeros((xyz.shape[0], len(rot_names)))
for idx, attr_name in enumerate(rot_names):
    rots[:, idx] = np.asarray(ply_data.elements[0][attr_name])

In [7]:
features_dc.shape

(2571949, 3, 1)

In [9]:
features_dc.min(), features_dc.max()

(-2.4952595233917236, 10.781548500061035)

In [11]:
features_dc.shape

(2571949, 3, 1)

In [15]:
features_dc[::1000].reshape(-1, 3)

array([[ 0.6154449 ,  0.74731845,  1.04085994],
       [-0.98088789, -0.83205801, -1.51029134],
       [-0.2435436 , -0.23995665,  0.07599381],
       ...,
       [ 0.03887702,  0.77560127,  1.36817038],
       [ 0.30363807,  0.05354311,  0.12016449],
       [ 1.52908432,  1.76625729,  1.60658956]])

In [29]:
X, Y, Z = xyz[::1000, :].T
colors = features_dc[::1000].reshape(-1, 3)

colors_formatted = list(map(
    lambda raw :  f"rgb({int(raw[0] * 255)}, {int(raw[1] * 255)}, {int(raw[2] * 255)})",
    colors
))

# Create a scatter3d plot
trace = go.Scatter3d(
    x=X,
    y=Y,
    z=Z,
    mode='markers',
    marker=dict(
        size=1 ,  # Change the size s]of markers here
        #color=colors_formatted,  # Set color to the Z coordinate for depth effect
        color = Z,
        #colorscale='Viridis',  # Choose a color scale
        opacity=0.8
    )
)

# Set up the layout of the plot
layout = go.Layout(
    margin=dict(l=0, r=0, b=0, t=0)
)

# Combine the plot and layout into a figure
fig = go.Figure(data=[trace], layout=layout)
fig.update_scenes(aspectmode='data')

# Render the plot
pyo.plot(fig)
iplot(fig)


In [23]:
from utils.sh_utils import SH2RGB

SH2RGB(features_dc).min(), SH2RGB(features_dc).max()

(-0.2038997156729747, 3.541418679124686)